In [72]:
import os
from tarefa02.gcrf import Gcrf, Point, Edge, Face

In [2]:
gc = Gcrf()

In [3]:
p = Point(1, 2, 3)
print(p)

(1, 2, 3)


In [4]:
p1 = Point(1, 2, 3)
p2 = Point(4, 5, 6)

e = Edge(p1, p2)

print(e)

[(1, 2, 3) -> (4, 5, 6)]


In [5]:
p1 = Point(1, 2, 3)
p2 = Point(4, 5, 6)
p3 = Point(7, 8, 9)

e1 = Edge(p1, p2)
e2 = Edge(p2, p3)
e3 = Edge(p3, p1)

f = Face(e1, e2, e3)
f_ = Face(e3, e1, e2)
print(f)

{[(1, 2, 3) -> (4, 5, 6)], [(4, 5, 6) -> (7, 8, 9)], [(7, 8, 9) -> (1, 2, 3)]}


In [6]:
def checa_face(F, f):
    for face in F:
        if f == face:
            return True
    return False

In [42]:
def PivotAroundEdge(e, P, H=[]):
    print("Entrada:", e)
    p = P[-1]
    area2 = gc.square_area(e.p1, e.p2, p)
    for p_ in range(len(P)-1):
        volume = gc.signed_volume(e.p1, e.p2, p, P[p_])
#         print("P",p)
#         print("P_", P[p_])
#         print("VOLUME", volume)
        face_no_hull = checa_face(H, Face(e, Edge(e.p1, P[p_]), Edge(P[p_], e.p2)))
#         print("FACE NO HULL", face_no_hull)
        if volume < 0 and not face_no_hull:
            p = P[p_]
        elif volume == 0:
            area2_ = gc.square_area(e.p1, e.p2, P[p_])
#             print(area2)
#             print(area2_)
            if area2_ > area2 and not face_no_hull:
                p = P[p_]
                area2 = area2_
    print("Saída:", p)
    return p

In [29]:
def BottomMostLeftMostBackMost(P):
    ys = [p.y for p in P]
    min_value = min(ys)
    min_index = ys.index(min_value)
    return P[min_index]

In [65]:
def FindEdgeOnHull(P):
    p = BottomMostLeftMostBackMost(P)
    q = p

    for r in P:
        if (q.z == r.z) and (q.y == r.y) and (q.x < r.x):
            q = r
    
    if (q == p):
        q = Point(p.x + 1, p.y, p.z)
#         print("Chamando PIVOT no Find Edge on Hull")
        q = PivotAroundEdge(Edge(p, q), P)

    return Edge(p, q)

In [66]:
def FindTriangleOnHull(P):
    e = FindEdgeOnHull(P)
    r = PivotAroundEdge(e, P)
    e2 = Edge(e.p2, r)
    e3 = Edge(r, e.p1)
    f = Face(e, e2, e3)
    return f

In [67]:
def NotProcessed(E, e):
#     print("Aresta no Not Processed",e)
    for i in E:
        if e == i:
            return False
    return True

In [68]:
def GiftWrap(P):
    t = FindTriangleOnHull(P)
    print("Triangulo encontrado:", t)
    Q = [t.e1, t.e2, t.e3]
    H = [t]
    Processed = [t.e1]

    while not len(Q) == 0:
        e = Q.pop()
#         print("Aresta a ser processada:", e)
        if NotProcessed(Processed, e):
            print()
#             print("Aresta {} não foi processada, entrando no IF".format(e))
            q = PivotAroundEdge(e, P, H)
#             print("Resultado do PivotAroundEdge:", q)
            print()
            t = Face(e, Edge(e.p2, q), Edge(q, e.p1))
#             print("Nova face:", t)
            face_no_hull = checa_face(H, t)
            if not face_no_hull:
                H.append(t)
#             print("{} entrou no Hull".format(t))
            Q.extend([t.e1, t.e2, t.e3])
#             print("Q agora possui as arestas:")
#             for q in Q: print(q)
            Processed.append(e)
#             print("Processed agora inclui a aresta {} e contém as seguintes arestas:".format(e))
#             for aresta in Processed: print(aresta)
#         else: print("Aresta JÁ PROCESSADA")
    return H

In [69]:
# p1 = Point(-3, 0, 0)
# p2 = Point(0, -3, 0)
# p3 = Point(0, 2, 0)
# p4 = Point(0, 0, 3)
# p5 = Point(0, 0, -4)
# p6 = Point(4, 0, 0)
# p7 = Point(0, 0, 0)

p1 = Point(-1, 0, 0)
p2 = Point(0, -1, 0)
p3 = Point(0, 1, 0)
p4 = Point(0, 0, 1)
p5 = Point(0, 0, -1)
p6 = Point(1, 0, 0)
p7 = Point(0, 0, 0)

P = [p1, p2, p3, p4, p5, p6, p7]

H = GiftWrap(P)
for h in H:
    print(h)
print(len(H))

Entrada: [(0, -1, 0) -> (1, -1, 0)]
Saída: (0, 0, -1)
Entrada: [(0, -1, 0) -> (0, 0, -1)]
Saída: (-1, 0, 0)
Triangulo encontrado: {[(0, -1, 0) -> (0, 0, -1)], [(0, 0, -1) -> (-1, 0, 0)], [(-1, 0, 0) -> (0, -1, 0)]}

Entrada: [(-1, 0, 0) -> (0, -1, 0)]
Saída: (0, 0, -1)


Entrada: [(0, 0, -1) -> (-1, 0, 0)]
Saída: (0, 0, 1)


Entrada: [(0, 0, 1) -> (0, 0, -1)]
Saída: (1, 0, 0)


Entrada: [(1, 0, 0) -> (0, 0, 1)]
Saída: (0, 1, 0)


Entrada: [(0, 1, 0) -> (1, 0, 0)]
Saída: (0, 0, 1)


Entrada: [(0, 0, 1) -> (0, 1, 0)]
Saída: (0, -1, 0)


Entrada: [(0, -1, 0) -> (0, 0, 1)]
Saída: (1, 0, 0)


Entrada: [(1, 0, 0) -> (0, -1, 0)]
Saída: (0, 0, 1)


Entrada: [(0, 1, 0) -> (0, -1, 0)]
Saída: (-1, 0, 0)


Entrada: [(-1, 0, 0) -> (0, 1, 0)]
Saída: (0, 0, 1)


Entrada: [(0, 0, 1) -> (-1, 0, 0)]
Saída: (0, 1, 0)


Entrada: [(0, 0, -1) -> (1, 0, 0)]
Saída: (0, 1, 0)


Entrada: [(0, 1, 0) -> (0, 0, -1)]
Saída: (1, 0, 0)

{[(0, -1, 0) -> (0, 0, -1)], [(0, 0, -1) -> (-1, 0, 0)], [(-1, 0, 0) -> (0, -1, 0

In [97]:
def write_faces(F, P, filename=False):
    i1 = 0
    i2 = 0
    i3 = 0
    if not filename:
        filename = os.path.join(os.getcwd(), 'faces.obj')
    with open(filename, 'w') as f:
        for point in P:
            f.write("v {} {} {}".format(point.x, point.y, point.z))
            f.write('\n')
        f.write('\n')
        for i in range(len(F)):
            for j in range(len(P)):
                if F[i]()[0][0] == P[j].x and F[i]()[0][1] == P[j].y and F[i]()[0][2] == P[j].z:
                    i1 = j
                if F[i]()[1][0] == P[j].x and F[i]()[1][1] == P[j].y and F[i]()[1][2] == P[j].z:
                    i2 = j
                if F[i]()[2][0] == P[j].x and F[i]()[2][1] == P[j].y and F[i]()[2][2] == P[j].z:
                    i3 = j
            f.writelines("f {} {} {}".format(i1+1, i2+1, i3+1))
            f.write('\n')


In [86]:
def print_faces(F, P):
    i1 = 0
    i2 = 0
    i3 = 0
    for i in range(len(F)):
        for j in range(len(P)):
            if F[i]()[0][0] == P[j].x and F[i]()[0][1] == P[j].y and F[i]()[0][2] == P[j].z:
                i1 = j
            if F[i]()[1][0] == P[j].x and F[i]()[1][1] == P[j].y and F[i]()[1][2] == P[j].z:
                i2 = j
            if F[i]()[2][0] == P[j].x and F[i]()[2][1] == P[j].y and F[i]()[2][2] == P[j].z:
                i3 = j
                
        print("f {} {} {}".format(i1+1, i2+1, i3+1))

In [87]:
def read_vertices_from_obj(objPath):
    points = []
    with open(objPath, 'r') as f:
        for line in f:
            line = line.replace('v', '')
            line_ = line.split()
            points.append(Point(float(line_[0]), float(line_[1]), float(line_[2])))
    return points

In [88]:
points = read_vertices_from_obj(r"C:\Users\romul\Documents\UFC\MESTRADO\Disciplinas\Geometria Computacional\super-nintendo-blend\p1.obj")
print(len(points))

348


In [90]:
nH = GiftWrap(points)
for h in nH:
    print(h)
print(len(nH))

Entrada: [(-0.258873, 0.032391, 0.264508) -> (0.741127, 0.032391, 0.264508)]
Saída: (-0.346055, 0.032391, 0.129129)
Entrada: [(-0.258873, 0.032391, 0.264508) -> (-0.346055, 0.032391, 0.129129)]
Saída: (-0.274323, 0.043444, 0.232575)
Triangulo encontrado: {[(-0.258873, 0.032391, 0.264508) -> (-0.346055, 0.032391, 0.129129)], [(-0.346055, 0.032391, 0.129129) -> (-0.274323, 0.043444, 0.232575)], [(-0.274323, 0.043444, 0.232575) -> (-0.258873, 0.032391, 0.264508)]}

Entrada: [(-0.274323, 0.043444, 0.232575) -> (-0.258873, 0.032391, 0.264508)]
Saída: (-0.319239, 0.044939, 0.161753)


Entrada: [(-0.319239, 0.044939, 0.161753) -> (-0.274323, 0.043444, 0.232575)]
Saída: (-0.321989, 0.044141, 0.158056)


Entrada: [(-0.321989, 0.044141, 0.158056) -> (-0.319239, 0.044939, 0.161753)]
Saída: (-0.274323, 0.043444, 0.232575)


Entrada: [(-0.274323, 0.043444, 0.232575) -> (-0.321989, 0.044141, 0.158056)]
Saída: (-0.365416, 0.047197, 0.088425)


Entrada: [(-0.365416, 0.047197, 0.088425) -> (-0.274323, 

In [98]:
print_faces(H, P)
print_faces(nH, points)
write_faces(H, P)

f 2 5 1
f 1 2 5
f 5 1 4
f 4 5 6
f 6 4 3
f 3 6 4
f 4 3 2
f 2 4 6
f 6 2 4
f 3 2 1
f 1 3 4
f 4 1 3
f 5 6 3
f 3 5 6
f 279 296 336
f 336 279 341
f 341 336 338
f 338 341 336
f 336 338 306
f 306 336 338
f 338 306 341
f 306 341 337
f 337 306 338
f 338 337 339
f 339 338 337
f 337 339 341
f 341 337 339
f 339 341 338
f 279 341 342
f 342 279 341
f 341 342 338
f 342 338 343
f 343 342 341
f 341 343 342
f 338 343 340
f 340 338 342
f 342 340 336
f 336 342 340
f 340 336 335
f 335 340 336
f 336 335 345
f 345 336 342
f 342 345 346
f 346 342 345
f 345 346 340
f 340 345 346
f 346 340 335
f 335 346 345
f 345 335 346
f 343 340 342
f 296 336 294
f 294 296 279
f 279 294 296
f 336 294 295
f 295 336 340
f 340 295 336
f 294 295 340
f 340 294 336
